In [1]:
shape_cat="mcal"
vol="vlim"
z_range="all_z"

lambda_range=0
with_r=True

In [2]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast
import gc

import astropy.cosmology
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from IPython.display import display, Math
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
#This header file contains all the common packages
%matplotlib inline
%run -i /home/adt35/des_y1_catalog/sa-master/lib/src/header.py $shape_cat $vol $z_range 
%run -i /home/adt35/des_y1_catalog/sa-master/lib/src/func.py  
%run -i /home/adt35/des_y1_catalog/sa-master/lib/src/corr_func.py

members= pd.read_pickle("/home/adt35/des_y1_catalog/sa-master/data/BPZ.pkl") #load BPZ catalog
red_members=pd.read_pickle("/home/adt35/des_y1_catalog/sa-master/data/shape_BPZ.pkl")
red_members_index=red_members.index
red_members=None

#uber=pd.read_pickle('/home/adt35/des_y1_catalog/sa-master/data/{}/{}/shapes{}.pkl'.format("mcal","uber",""))
vlim=pd.read_pickle('/home/adt35/des_y1_catalog/sa-master/data/{}/{}/shapes{}.pkl'.format("mcal","vlim",""))

# foreground background

In [ ]:
def get_ng_source(_clusters,_sources,_shapes,z_lower=z_bins[0],z_upper=z_bins[-1],lambda_lower=lambda_bins[0],lambda_upper=lambda_bins[-1],foreback=False,woRed=False,multi=True):
#     print(len(_clusters))

#     print(_clusters)
    
    if multi==False:
        assert len(_clusters)==1
        clusters_masked=_clusters
    else:
        assert len(_clusters)!=1
        clusters_z_mask=(_clusters[('All','All','Z_LAMBDA')]>=z_lower)&(_clusters[('All','All','Z_LAMBDA')]<=z_upper)
        clusters_lambda_mask=(_clusters[('All','All','LAMBDA_CHISQ')]>=lambda_lower)&(_clusters[('All','All','LAMBDA_CHISQ')]<=lambda_upper)
        combined_mask=clusters_z_mask&clusters_lambda_mask
        clusters_masked=_clusters[combined_mask]
        
        
    
    
    if woRed==True:
        _sources=_sources[woRedMask]
        
        
        
    center_id=clusters_masked[('Alt','Alt1','ID_CENT')]
    centers=_shapes.loc[center_id]
    
#     print(center_id)
#     print(centers)
    
    
    
    
    assert ~np.any(np.isnan(centers[('All','RA')]))
    
#     print("The number of clusters is {}".format(len(clusters_masked)))
#     print("The number of sources is {}".format(len(sources)))
    
    
    UPPER_BOUND=10
    distance=('All','angR')


#     if mode1 == "s":
#         def get_cluster_cen(cluster):
#             return (get_unique_center_for_cluster(cluster))
#         def get_cluster_sats(cluster):
#             cen_mat_id=cluster.name
#             cen_id=cluster[('Alt','Alt1','ID_CENT')]
            

            
#             if source==True:
#                 sats=members
                
#             else: 
#                 sats=members[members['All','MEM_MATCH_ID']==cen_mat_id]
                         
#             sats=sats[sats.index!=cen_id]
#             return (sats)
#     elif mode1=='r':
#         def get_cluster_sats(cluster):
#             return(shapes)
#         def get_cluster_cen(cluster):
#             return (random)

#     else:
#         raise


    cen=centers
    sats=sources
    
    if foreback=="back":
        sats=sats[(sats[('All','mean_z')]-z_upper)>=0.1]
        print("The number of background sources is {}".format(len(sats)))
    elif foreback=="fore":
        sats=sats[(z_lower-sats[('All','mean_z')])>=0.1]
        print("Calculating foreground sources")
        print(sats[('All','mean_z')].mean())
        print("The number of foreground sources is {}".format(len(sats)))

    sats_e1=sats[('All','e1')].to_numpy()
    sats_e2=sats[('All','e2')].to_numpy()
    sats_angr=sats[distance]
    

#     print(sats_e1,sats_e2)

    if type(cen[('All','RA')])==np.float64:
        cen_angr=np.array(cen[distance]).reshape(1)
        cen_ra=np.array(cen[('All','RA')]).reshape(1)
        cen_dec=np.array(cen[('All','DEC')]).reshape(1)
    else:
#         cen_e1=cen[('All','e1')].to_numpy
#         cen_e2=cen[('All','e2')].to_numpy()
        cen_angr=cen[distance].to_numpy()
        cen_ra=cen[('All','RA')].to_numpy()
        cen_dec=cen[('All','DEC')].to_numpy()
        




#     print(cen_e1,cen_e2)

    sats_ra=sats[('All','RA')].to_numpy()
    sats_dec=sats[('All','DEC')].to_numpy()




#     print(np.shape(cen_ra),np.shape(cen_dec))    
#     print(cen_ra,cen_dec)    

    sats_cat = treecorr.Catalog( g1 = sats_e1, g2   = sats_e2, 
                                 ra = sats_ra, dec = sats_dec,r=sats_angr,
                                 ra_units='deg', dec_units='deg')

    cen_cat = treecorr.Catalog( 
                                 ra = cen_ra, dec = cen_dec, r=cen_angr,
                                 ra_units='deg', dec_units='deg')
    
    MINSEP=0.1
    BINSLOP=0.1
        

    ng = treecorr.NGCorrelation(nbins=NBINS, min_sep=MINSEP, max_sep=UPPER_BOUND, bin_slop=BINSLOP,
                               metric="Rperp")
    

    ng.process(cen_cat,sats_cat)

    return(ng)

In [4]:
ng_matrix_fore=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],foreback="fore")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_matrix_fore[i][j]=ng_values
        del ng
        gc.collect()

NameError: name 'z_bins' is not defined

In [ ]:
fig, axs = plt.subplots(3, 4,figsize=(20,15))
for i in range(3):
    for j in range(4):
        values=ng_matrix_fore[i][j]
        print((i,j),values[1])
        axs[i,j].plot(np.exp(values[0]),values[1])
        axs[i,j].set_xscale('log')
        axs[i,j].set_yscale('log')

In [ ]:
ng_matrix_back=[[None for i in range(4)] for j in range(3)]

for i in trange(3):
    for j in trange(4):
        ng=get_ng_source(clusters,members,z_bins[i],z_bins[i+1],lambda_bins[j],lambda_bins[j+1],foreback="")
        ng_values=np.copy(np.array([ng.meanlogr,ng.xi,ng.varxi]))
        ng_matrix_back[i][j]=ng_values
        del ng
        gc.collect()

In [ ]:
fig, axs = plt.subplots(3, 4,figsize=(20,15))
for i in range(3):
    for j in range(4):
        values=ng_matrix_fore[i][j]
        print((i,j),values[0])
        axs[i,j].plot(np.exp(values[0]),values[1])
        axs[i,j].set_xscale('log')
        axs[i,j].set_yscale('log')